In [3]:
from collections import defaultdict
import json
import os
import pickle
import statistics

from requests import Session

import matplotlib.pyplot as plt
import pandas as pd

import statbotics

%matplotlib notebook

In [4]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

sb = statbotics.Statbotics()

In [5]:
def dump(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path, "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load(file):
    with open(file, "rb") as f:
        return pickle.load(f)


def dump_cache(path, data):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path + "/data.p", "wb") as f:
            pickle.dump(data, f)
    except OSError:
        pass


def load_cache(file):
    with open(file + "/data.p", "rb") as f:
        return pickle.load(f)

    
    
def get_tba(url):
    if os.path.exists("../../backend/cache/" + url + "/data.p"):
        # Cache Hit
        return load_cache("../../backend/cache/" + url)

    response = session.get(read_prefix + url)
    data = response.json()

    # Cache Miss
    dump_cache("../../backend/cache/" + url, data)
    return data

In [6]:
events = get_tba("events/2023")
events = [(e["key"], e["week"]) for e in events]

print(len(events))

all_matches = []
for event, week in events:
    all_matches.extend([x, week] for x in get_tba("event/" + event + "/matches"))
    
print(len(all_matches))

179
10449


In [7]:
year = sb.get_year(2023)
teams = sb.get_team_years(year=2023, limit=10000)

TOTAL_MEAN, TOTAL_SD = year["score_mean"], year["score_sd"]
starting_epas = {t["team"]: (t["epa_start"] - TOTAL_MEAN / 3) / TOTAL_SD for t in teams}
locations = {t["team"]: (t["country"], t["state"], t["district"]) for t in teams}
print(starting_epas[254], starting_epas[5511])

0.9612851952770209 0.03689827429609454


In [29]:
def process_match(m, w):
    rb = m["score_breakdown"]["red"]
    bb = m["score_breakdown"]["blue"]
    
    out = {
        "week": w + 1, # TBA off by one
        "event": m["event_key"],
        "key": m["key"],
        "time": m["actual_time"],
        "playoff": m["comp_level"] != "qm",
        "red": [int(x[3:]) for x in m["alliances"]["red"]["team_keys"]],
        "blue": [int(x[3:]) for x in m["alliances"]["blue"]["team_keys"]],
        "red_auto_cycles": rb["autoGamePieceCount"],
        "blue_auto_cycles": bb["autoGamePieceCount"],
        "red_auto_points": rb["autoGamePiecePoints"],
        "blue_auto_points": bb["autoGamePiecePoints"],
        "red_teleop_cycles": rb["teleopGamePieceCount"],
        "blue_teleop_cycles": bb["teleopGamePieceCount"],
        "red_teleop_points": rb["teleopGamePiecePoints"],
        "blue_teleop_points": bb["teleopGamePiecePoints"],
        "red_links": rb["linkPoints"] / 5,
        "blue_links": bb["linkPoints"] / 5,
        "red_link_points": rb["linkPoints"],
        "blue_link_points": bb["linkPoints"],
        "red_total_cycles": rb["autoGamePieceCount"] + rb["teleopGamePieceCount"],
        "blue_total_cycles": bb["autoGamePieceCount"] + bb["teleopGamePieceCount"],
        "red_total_points": rb["autoGamePiecePoints"] + rb["teleopGamePiecePoints"] + rb["linkPoints"],
        "blue_total_points": bb["autoGamePiecePoints"] + bb["teleopGamePiecePoints"] + bb["linkPoints"],
        "red_auto_bot_cubes": len([x for x in rb["autoCommunity"]["B"] if x == "Cube"]),
        "blue_auto_bot_cubes": len([x for x in bb["autoCommunity"]["B"] if x == "Cube"]),
        "red_auto_mid_cubes": len([x for x in rb["autoCommunity"]["M"] if x == "Cube"]),
        "blue_auto_mid_cubes": len([x for x in bb["autoCommunity"]["M"] if x == "Cube"]),
        "red_auto_top_cubes": len([x for x in rb["autoCommunity"]["T"] if x == "Cube"]),
        "blue_auto_top_cubes": len([x for x in bb["autoCommunity"]["T"] if x == "Cube"]),
        "red_auto_bot_cones": len([x for x in rb["autoCommunity"]["B"] if x == "Cone"]),
        "blue_auto_bot_cones": len([x for x in bb["autoCommunity"]["B"] if x == "Cone"]),
        "red_auto_mid_cones": len([x for x in rb["autoCommunity"]["M"] if x == "Cone"]),
        "blue_auto_mid_cones": len([x for x in bb["autoCommunity"]["M"] if x == "Cone"]),
        "red_auto_top_cones": len([x for x in rb["autoCommunity"]["T"] if x == "Cone"]),
        "blue_auto_top_cones": len([x for x in bb["autoCommunity"]["T"] if x == "Cone"]),
        "red_auto_bot_cycles": len([x for x in rb["autoCommunity"]["B"] if x != "None"]),
        "blue_auto_bot_cycles": len([x for x in bb["autoCommunity"]["B"] if x != "None"]),
        "red_auto_mid_cycles": len([x for x in rb["autoCommunity"]["M"] if x != "None"]),
        "blue_auto_mid_cycles": len([x for x in bb["autoCommunity"]["M"] if x != "None"]),
        "red_auto_top_cycles": len([x for x in rb["autoCommunity"]["T"] if x != "None"]),
        "blue_auto_top_cycles": len([x for x in bb["autoCommunity"]["T"] if x != "None"]),
        
        "red_teleop_bot_cubes": len([x for x in rb["teleopCommunity"]["B"] if x == "Cube"]),
        "blue_teleop_bot_cubes": len([x for x in bb["teleopCommunity"]["B"] if x == "Cube"]),
        "red_teleop_mid_cubes": len([x for x in rb["teleopCommunity"]["M"] if x == "Cube"]),
        "blue_teleop_mid_cubes": len([x for x in bb["teleopCommunity"]["M"] if x == "Cube"]),
        "red_teleop_top_cubes": len([x for x in rb["teleopCommunity"]["T"] if x == "Cube"]),
        "blue_teleop_top_cubes": len([x for x in bb["teleopCommunity"]["T"] if x == "Cube"]),
        "red_teleop_bot_cones": len([x for x in rb["teleopCommunity"]["B"] if x == "Cone"]),
        "blue_teleop_bot_cones": len([x for x in bb["teleopCommunity"]["B"] if x == "Cone"]),
        "red_teleop_mid_cones": len([x for x in rb["teleopCommunity"]["M"] if x == "Cone"]),
        "blue_teleop_mid_cones": len([x for x in bb["teleopCommunity"]["M"] if x == "Cone"]),
        "red_teleop_top_cones": len([x for x in rb["teleopCommunity"]["T"] if x == "Cone"]),
        "blue_teleop_top_cones": len([x for x in bb["teleopCommunity"]["T"] if x == "Cone"]),
        "red_teleop_bot_cycles": len([x for x in rb["teleopCommunity"]["B"] if x != "None"]),
        "blue_teleop_bot_cycles": len([x for x in bb["teleopCommunity"]["B"] if x != "None"]),
        "red_teleop_mid_cycles": len([x for x in rb["teleopCommunity"]["M"] if x != "None"]),
        "blue_teleop_mid_cycles": len([x for x in bb["teleopCommunity"]["M"] if x != "None"]),
        "red_teleop_top_cycles": len([x for x in rb["teleopCommunity"]["T"] if x != "None"]),
        "blue_teleop_top_cycles": len([x for x in bb["teleopCommunity"]["T"] if x != "None"]),
    }
    
    out["red_auto_cube_cycles"] = out["red_auto_bot_cubes"] + out["red_auto_mid_cubes"] + out["red_auto_top_cubes"]
    out["blue_auto_cube_cycles"] = out["blue_auto_bot_cubes"] + out["blue_auto_mid_cubes"] + out["blue_auto_top_cubes"]
    out["red_auto_cone_cycles"] = out["red_auto_bot_cones"] + out["red_auto_mid_cones"] + out["red_auto_top_cones"]
    out["blue_auto_cone_cycles"] = out["blue_auto_bot_cones"] + out["blue_auto_mid_cones"] + out["blue_auto_top_cones"]
    out["red_auto_cube_points"] = 3 * out["red_auto_bot_cubes"] + 4 * out["red_auto_mid_cubes"] + 6 * out["red_auto_top_cubes"]
    out["blue_auto_cube_points"] = 3 * out["blue_auto_bot_cubes"] + 4 * out["blue_auto_mid_cubes"] + 6 * out["blue_auto_top_cubes"]
    out["red_auto_cone_points"] = 3 * out["red_auto_bot_cones"] + 4 * out["red_auto_mid_cones"] + 6 * out["red_auto_top_cones"]
    out["blue_auto_cone_points"] = 3 * out["blue_auto_bot_cones"] + 4 * out["blue_auto_mid_cones"] + 6 * out["blue_auto_top_cones"]
    
    out["red_teleop_cube_cycles"] = out["red_teleop_bot_cubes"] + out["red_teleop_mid_cubes"] + out["red_teleop_top_cubes"]
    out["blue_teleop_cube_cycles"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_mid_cubes"] + out["blue_teleop_top_cubes"]
    out["red_teleop_cone_cycles"] = out["red_teleop_bot_cones"] + out["red_teleop_mid_cones"] + out["red_teleop_top_cones"]
    out["blue_teleop_cone_cycles"] = out["blue_teleop_bot_cones"] + out["blue_teleop_mid_cones"] + out["blue_teleop_top_cones"]
    out["red_teleop_cube_points"] = 2 * out["red_teleop_bot_cubes"] + 3 * out["red_teleop_mid_cubes"] + 5 * out["red_teleop_top_cubes"]
    out["blue_teleop_cube_points"] = 2 * out["blue_teleop_bot_cubes"] + 3 * out["blue_teleop_mid_cubes"] + 5 * out["blue_teleop_top_cubes"]
    out["red_teleop_cone_points"] = 2 * out["red_teleop_bot_cones"] + 3 * out["red_teleop_mid_cones"] + 5 * out["red_teleop_top_cones"]
    out["blue_teleop_cone_points"] = 2 * out["blue_teleop_bot_cones"] + 3 * out["blue_teleop_mid_cones"] + 5 * out["blue_teleop_top_cones"]
    
    out["red_cube_cycles"] = out["red_auto_cube_cycles"] + out["red_teleop_cube_cycles"]
    out["blue_cube_cycles"] = out["blue_auto_cube_cycles"] + out["blue_teleop_cube_cycles"]
    out["red_cone_cycles"] = out["red_auto_cone_cycles"] + out["red_teleop_cone_cycles"]
    out["blue_cone_cycles"] = out["blue_auto_cone_cycles"] + out["blue_teleop_cone_cycles"]
    out["red_cube_points"] = out["red_auto_cube_points"] + out["red_teleop_cube_points"]
    out["blue_cube_points"] = out["blue_auto_cube_points"] + out["blue_teleop_cube_points"]
    out["red_cone_points"] = out["red_auto_cone_points"] + out["red_teleop_cone_points"]
    out["blue_cone_points"] = out["blue_auto_cone_points"] + out["blue_teleop_cone_points"]
    
    out["red_auto_bot_cycles"] = out["red_auto_bot_cubes"] + out["red_auto_bot_cones"]
    out["blue_auto_bot_cycles"] = out["blue_auto_bot_cubes"] + out["blue_auto_bot_cones"]
    out["red_auto_mid_cycles"] = out["red_auto_mid_cubes"] + out["red_auto_mid_cones"]
    out["blue_auto_mid_cycles"] = out["blue_auto_mid_cubes"] + out["blue_auto_mid_cones"]
    out["red_auto_top_cycles"] = out["red_auto_top_cubes"] + out["red_auto_top_cones"]
    out["blue_auto_top_cycles"] = out["blue_auto_top_cubes"] + out["blue_auto_top_cones"]
    out["red_auto_bot_points"] = 3 * out["red_auto_bot_cycles"]
    out["blue_auto_bot_points"] = 3 * out["blue_auto_bot_cycles"]
    out["red_auto_mid_points"] = 4 * out["red_auto_mid_cycles"]
    out["blue_auto_mid_points"] = 4 * out["blue_auto_mid_cycles"]
    out["red_auto_top_points"] = 6 * out["red_auto_top_cycles"]
    out["blue_auto_top_points"] = 6 * out["blue_auto_top_cycles"]
    
    out["red_teleop_bot_cycles"] = out["red_teleop_bot_cubes"] + out["red_teleop_bot_cones"]
    out["blue_teleop_bot_cycles"] = out["blue_teleop_bot_cubes"] + out["blue_teleop_bot_cones"]
    out["red_teleop_mid_cycles"] = out["red_teleop_mid_cubes"] + out["red_teleop_mid_cones"]
    out["blue_teleop_mid_cycles"] = out["blue_teleop_mid_cubes"] + out["blue_teleop_mid_cones"]
    out["red_teleop_top_cycles"] = out["red_teleop_top_cubes"] + out["red_teleop_top_cones"]
    out["blue_teleop_top_cycles"] = out["blue_teleop_top_cubes"] + out["blue_teleop_top_cones"]
    out["red_teleop_bot_points"] = 2 * out["red_teleop_bot_cycles"]
    out["blue_teleop_bot_points"] = 2 * out["blue_teleop_bot_cycles"]
    out["red_teleop_mid_points"] = 3 * out["red_teleop_mid_cycles"]
    out["blue_teleop_mid_points"] = 3 * out["blue_teleop_mid_cycles"]
    out["red_teleop_top_points"] = 5 * out["red_teleop_top_cycles"]
    out["blue_teleop_top_points"] = 5 * out["blue_teleop_top_cycles"]
    
    out["red_bot_cycles"] = out["red_auto_bot_cycles"] + out["red_teleop_bot_cycles"]
    out["blue_bot_cycles"] = out["blue_auto_bot_cycles"] + out["blue_teleop_bot_cycles"]
    out["red_mid_cycles"] = out["red_auto_mid_cycles"] + out["red_teleop_mid_cycles"]
    out["blue_mid_cycles"] = out["blue_auto_mid_cycles"] + out["blue_teleop_mid_cycles"]
    out["red_top_cycles"] = out["red_auto_top_cycles"] + out["red_teleop_top_cycles"]
    out["blue_top_cycles"] = out["blue_auto_top_cycles"] + out["blue_teleop_top_cycles"]
    out["red_bot_points"] = out["red_auto_bot_points"] + out["red_teleop_bot_points"]
    out["blue_bot_points"] = out["blue_auto_bot_points"] + out["blue_teleop_bot_points"]
    out["red_mid_points"] = out["red_auto_mid_points"] + out["red_teleop_mid_points"]
    out["blue_mid_points"] = out["blue_auto_mid_points"] + out["blue_teleop_mid_points"]
    out["red_top_points"] = out["red_auto_top_points"] + out["red_teleop_top_points"]
    out["blue_top_points"] = out["blue_auto_top_points"] + out["blue_teleop_top_points"]
    
    return out

processed_matches = sorted([process_match(m, w) for m, w in all_matches if m["score_breakdown"] is not None and w is not None], key=lambda x: x["time"])
week_one_matches = [m for m in processed_matches if m["week"] == 1]

In [30]:
keys = [
    "auto_cycles",
    "auto_points",
    "teleop_cycles",
    "teleop_points",
    "total_cycles",
    "links",
    "link_points",
    "total_points",
    "cube_cycles",
    "cube_points",
    "cone_cycles",
    "cone_points",
    "bot_cycles",
    "bot_points",
    "mid_cycles",
    "mid_points",
    "top_cycles",
    "top_points",
]

team_epas = defaultdict(dict)

for key in keys:
    stats = [m["red_" + key] for m in week_one_matches] + [m["blue_" + key] for m in week_one_matches]
    mean, sd = sum(stats) / len(stats), statistics.stdev(stats)

    for team in starting_epas:
        team_epas[team][key] = [mean / 3 + sd * starting_epas[team]]

    for match in processed_matches:
        if match["playoff"]:
            continue

        red_teams = match["red"]
        blue_teams = match["blue"]

        red_pred = sum(team_epas[x][key][-1] for x in red_teams)
        blue_pred = sum(team_epas[x][key][-1] for x in blue_teams)

        red_actual = match["red_" + key]
        blue_actual = match["blue_" + key]

        red_error = red_actual - red_pred
        for team in red_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            team_epas[team][key].append(team_epas[team][key][-1] + percent * red_error / 3)

        blue_error = blue_actual - blue_pred
        for team in blue_teams:
            percent = min(0.5, max(0.3, 0.5 - 0.2 / 6 * (len(team_epas[team][key]) - 6)))
            team_epas[team][key].append(team_epas[team][key][-1] + percent * blue_error / 3)
        
end_epas = {
    k: {
    "country": locations[k][0],
    "state": locations[k][1],
    "district": locations[k][2],
    } 
    for k in team_epas
}

for k, v in team_epas.items():
    for k2, v2 in v.items():
        end_epas[k][k2] = v2[-1]
    
print(end_epas[254])

{'country': 'USA', 'state': 'CA', 'district': None, 'auto_cycles': 2.2102420030107646, 'auto_points': 11.98691842900499, 'teleop_cycles': 10.58280504223382, 'teleop_points': 31.14544949644772, 'total_cycles': 12.735670288506093, 'links': 3.2280904441405234, 'link_points': 16.140452220702617, 'total_points': 58.7677236147586, 'cube_cycles': 6.155258652209206, 'cube_points': 21.490564247923455, 'cone_cycles': 6.842067807462255, 'cone_points': 31.675933663975236, 'bot_cycles': 3.7328987270885494, 'bot_points': 7.876698731677259, 'mid_cycles': 1.8071348941940388, 'mid_points': 5.514918881009611, 'top_cycles': 7.820265100553488, 'top_points': 40.87202462029999}


In [31]:
df = pd.DataFrame(end_epas).T

In [32]:
df.head()

,country,state,district,auto_cycles,auto_points,teleop_cycles,teleop_points,total_cycles,links,link_points,...,cube_cycles,cube_points,cone_cycles,cone_points,bot_cycles,bot_points,mid_cycles,mid_points,top_cycles,top_points
1,USA,MI,fim,0.875079,4.949726,3.681288,10.07452,4.545365,0.973287,4.866433,...,3.671203,13.829163,0.924333,5.571789,1.73599,3.562553,-0.303052,-0.985554,3.232196,17.034324
4,USA,CA,None,0.912817,2.741662,3.633321,9.654981,4.539866,0.875898,4.379491,...,1.02186,3.079473,3.546608,11.097883,2.247973,5.421957,1.459713,4.397474,0.900458,4.477856
8,USA,CA,None,0.937661,5.763065,4.245342,11.485198,5.160951,0.609034,3.045169,...,3.812566,16.940358,1.448948,5.267921,1.012554,1.970697,1.150031,3.476206,3.23843,17.183043
11,USA,NJ,fma,0.973075,5.664309,6.176185,20.702562,7.146182,1.757148,8.785738,...,2.085003,7.826009,5.075212,23.224671,1.33468,2.650702,0.880699,2.768957,4.964302,25.68986
16,USA,AR,None,1.25185,7.566783,5.258693,12.796794,6.485824,1.9021,9.510501,...,2.066005,6.501998,4.532544,20.156712,2.134414,4.270189,0.759897,2.336252,3.860612,20.524936


In [33]:
df.to_json("../../backend/data/epa_breakdown.json", orient="index")